In [9]:
#Load all libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (8,5)

In [10]:
# Load cleaned dataset
df = pd.read_csv("../Data Cleaning/Processed/final_dataset.csv")

print("Shape:", df.shape)
df.head()

Shape: (14004, 42)


,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,carrier_lg,...,orig_overseas_visitation_2023,dest_state_name,dest_overseas_share_2024,dest_overseas_visitation_2024,dest_overseas_change_2024_vs_2023,dest_overseas_share_2023,dest_overseas_visitation_2023,fare_per_mile,dominance_bucket,lcc_bucket
0,2022,1,30135,34761,"Allentown/Bethlehem/Easton, PA","Sanford, FL",882,376.0,102.49,G4,...,843000.0,Florida,0.2520,8860000.0,0.117,0.2521,7933000.0,0.116202,dominated,high_lcc
1,2022,1,30140,30194,"Albuquerque, NM","Dallas/Fort Worth, TX",580,464.0,178.62,AA,...,85000.0,Texas,0.0594,2088000.0,0.115,0.0595,1872000.0,0.307966,moderate,high_lcc
2,2022,1,30140,30325,"Albuquerque, NM","Denver, CO",349,323.0,142.07,WN,...,85000.0,Colorado,0.0131,461000.0,-0.011,0.0148,466000.0,0.407077,moderate,high_lcc
3,2022,1,30140,30423,"Albuquerque, NM","Austin, TX",619,218.0,169.01,WN,...,85000.0,Texas,0.0594,2088000.0,0.115,0.0595,1872000.0,0.273037,moderate,high_lcc
4,2022,1,30140,30466,"Albuquerque, NM","Phoenix, AZ",328,481.0,165.33,WN,...,85000.0,Arizona,0.0330,1160000.0,0.233,0.0299,941000.0,0.504055,dominated,high_lcc


Data cleaning
- Drop rows with missing key variables
- Force all variables to be numeric
- Clip market share to be 0 and 1
- Create new variables: fare_per_mile and time_index

In [11]:

d = df.dropna(subset=["fare","nsmiles","passengers","large_ms","lf_ms"]).copy()

cols = ["fare","nsmiles","passengers","large_ms","lf_ms"]
for c in cols:
    d[c] = pd.to_numeric(d[c], errors="coerce")

d["large_ms"] = d["large_ms"].clip(0,1)
d["lf_ms"] = d["lf_ms"].clip(0,1)

d["fare_per_mile"] = d["fare"] / d["nsmiles"]
d["time_index"] = d["Year"]*4 + d["quarter"]

print("Cleaned shape:", d.shape)
d.info()

Cleaned shape: (14004, 43)
<class 'pandas.DataFrame'>
RangeIndex: 14004 entries, 0 to 14003
Data columns (total 43 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Year                               14004 non-null  int64  
 1   quarter                            14004 non-null  int64  
 2   citymarketid_1                     14004 non-null  int64  
 3   citymarketid_2                     14004 non-null  int64  
 4   city1                              14004 non-null  str    
 5   city2                              14004 non-null  str    
 6   nsmiles                            14004 non-null  int64  
 7   passengers                         14004 non-null  float64
 8   fare                               14004 non-null  float64
 9   carrier_lg                         14004 non-null  str    
 10  large_ms                           14004 non-null  float64
 11  fare_lg                            140

Feature Engineering
- log transformation for variables
- create net power and comp strength index to interpret dominance and competitiveness

In [13]:
import numpy as np

d["log_fare"] = np.log(d["fare"])
d["log_distance"] = np.log(d["nsmiles"])
d["log_passengers"] = np.log(d["passengers"] + 1)

In [14]:
d["market_power_index"] = d["large_ms"] - d["lf_ms"]
d["competition_strength"] = 1 - d["large_ms"]


Hub: This feature measures how strongly a route is connected to major airline hubs, capturing structural pricing power beyond simple competition metrics. Helps evaluate market power that is not captured by market share alone.

In [17]:
city_pax = (
    d.groupby("city1")["passengers"].sum()
    + d.groupby("city2")["passengers"].sum()
)

hub_threshold = city_pax.quantile(0.90)
hub_cities = set(city_pax[city_pax >= hub_threshold].index)

d["hub_intensity"] = (
    d["city1"].isin(hub_cities).astype(int)
    + d["city2"].isin(hub_cities).astype(int)
)

Fair premium vs discount: Measures how much a route’s fare exceeds what we would expect based purely on distance. It important cus it helps isolate price beyond cost, it represents a strong fairness and marketing indicator.

Passenger density: Measure demand intensity per mile. It help tests whether busy short routes command higher prices (demand-based pricing).

Time: Improves interpretability of time effects in regression.

In [ ]:
baseline = smf.ols("fare ~ nsmiles", data=d).fit()
d["distance_adjusted_fare"] = baseline.resid
d["passenger_density"] = d["passengers"] / d["nsmiles"]
d["years_since_2021"] = d["Year"] - 2021